In [ ]:
# import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
# setting
plt.rcParams['font.family']= 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['font.size'] = 18
plt.rcParams["figure.dpi"] = 200

In [ ]:
# List of H3K4me2 diel oscillating genes (DOGs)
file_path = f"../data/list_of_DOGs/H3K4me2_DOGs.bed"
H3K4me2_DOGs = pd.read_csv(file_path,sep="\t",header=None).iloc[:,3].values

In [ ]:
# Time-course ChIP-seq data in WT and atxr3
file_path = "../data/ChIP-seq/Time-course-ChIP_WT_atxr3_rep1.rpkm.tsv"
df_chip = pd.read_csv(file_path, sep="\t", index_col=0)

# Time-course mRNA-seq data in WT and atxr3
file_path = "../data/RNA-seq/Time-course-RNA_WT_atxr3.rpkm.tsv"
df_rna = pd.read_csv(file_path, sep="\t", index_col=0)
# mean of three replicates
gt_list = ["WT","atxr3"]
df_rna_mean = np.sqrt(pd.DataFrame({f"{gt_list[i]}_mRNA_ZT{6*j}":df_rna.iloc[:,12*i+3*j:12*i+3*(j+1)].mean(axis=1) for i in range(2) for j in range(4)}))

In [ ]:
df_all = df_chip.merge(df_rna_mean,left_index=True,right_index=True)
df_all.head()

In [ ]:
# Supplementary Fig. 10a-d
tp_list = ["ZT0","ZT6","ZT12","ZT18","ZT0"]
c_list = ["black","tab:orange","tab:cyan","tab:red","tab:purple"]

fig, ax = plt.subplots(5,4,figsize=(20,30))

for i,hm in enumerate(["H3","H3K4me2","H3K4me3","H2A.Z","mRNA"]):
    mask = df_all.index.isin(H3K4me2_DOGs)
    print(hm,mask.sum())
    for j in range(4):
        tp1 = tp_list[j]
        tp2 = tp_list[j+1]
        x = (df_all[f"WT_{hm}_{tp2}"] - df_all[f"WT_{hm}_{tp1}"])
        y = (df_all[f"atxr3_{hm}_{tp2}"] - df_all[f"atxr3_{hm}_{tp1}"])

        
        # liner regression
        slope, intercept, r_value, p_value, std_err = stats.linregress(x[mask], y[mask])

        # plot
        sns.regplot(x=x[mask],y=y[mask],ax=ax[i][j],color=c_list[i],
                    scatter_kws={"s":5, "color":"grey"})

        # format figure
        xy = np.min([np.percentile(x[mask],99),-np.percentile(x[mask],1)])
        ax[i][j].set_xlim(-xy,xy)
        ax[i][j].set_ylim(-xy,xy)
        ax[i][j].vlines(0,-xy,xy,color="k",linestyle="--")
        ax[i][j].hlines(0,-xy,xy,color="k",linestyle="--")
        ax[i][j].set_aspect('equal', adjustable='box')
        ax[i][j].text(x=xy*-0.97,y=xy*0.75,
                      s=f"$y={slope:.2f}x{intercept:+.2f}$\n$R^2={r_value**2:.2f}$",
                      c=c_list[i],fontsize=14)

In [ ]:
# Supplementary Fig. 10e
mask = df_all.index.isin(H3K4me2_DOGs)
df_tmp = df_all[mask]

# H3K4me2とH2A.Z
hm_list = ["H3K4me2","H3K4me3"]
for k in range(2):
    hm1 = "H2A.Z"
    hm2 = hm_list[k]
    print(hm1,hm2)

    # Calculation of Spearman's R
    R_matrix = np.zeros([4,4])
    for i in range(4):
        for j in range(4):
            x = df_tmp[f"atxr3_{hm2}_ZT{6*j}"] / df_tmp[f"WT_{hm2}_ZT{6*j}"]
            y = df_tmp[f"atxr3_{hm1}_ZT{6*i}"] / df_tmp[f"WT_{hm1}_ZT{6*i}"]
            R_matrix[i][j] = spearmanr(x,y)[0]

    # plot
    plt.figure(figsize=(8,5))
    sns.heatmap(R_matrix, cmap="bwr", center=0,vmax=0.4,vmin=-0.4,
                square=True, annot=True, fmt='.3f',
                linewidths=.5)
    plt.xticks([])
    plt.yticks([])